In [2]:
from tkinter import Frame, Canvas, filedialog, Menu, Tk, E
import abc as ABC
import socket
import threading
import json
from tkinter import Label
# Kreiranje Klase Graflik unutar koje ćemo raditi s bojama i pozicijama točaka.
# Dodana abstract funkcija za crtanje sljedećih klasa koje ju izvode
CLIENT_LIMIT = 10

        # List to keep track of active connections
clients = []
class Graflik(ABC.ABC):

    def __init__(self, boja, tocka):
        super().__init__()
        self.boja = boja
        self.tocka = tocka

    def SetColor(self, boja):
        self.boja = boja

    def GetColor(self):
        return self.boja

    @ABC.abstractmethod
    def Draw(self, Canvas):
        pass


class Linija(Graflik):

    def __init__(self, boja, tocka, tockaDva):
        Graflik.__init__(self, boja, tocka)
        self.tockaDva = tockaDva

    def Draw(self, Canvas):
        Canvas.create_line(
            self.tocka[0], self.tocka[1], self.tockaDva[0], self.tockaDva[1], fill=self.boja)


class Trokut(Linija):
    def __init__(self, boja, tocka,  tockaDva, tockaTri):
        Linija.__init__(self, boja, tocka, tockaDva)
        self.tockaTri = tockaTri

    def Draw(self, Canvas):
        Canvas.create_polygon(self.tocka[0], self.tocka[1], self.tockaDva[0], self.tockaDva[1],
                              self.tockaTri[0], self.tockaTri[1], fill="", outline=self.boja)


class Pravokutnik(Graflik):
    def __init__(self, boja, tocka, visina, sirina):
        Graflik.__init__(self, boja, tocka)
        self.visina = visina
        self.sirina = sirina

    def Draw(self, Canvas):
        Canvas.create_rectangle(self.tocka[0], self.tocka[1], self.tocka[0] +
                                self.sirina, self.tocka[1] + self.visina, fill="", outline=self.boja)


class Poligon(Graflik):
    def __init__(self, boja, tocka, liste):
        Graflik.__init__(self, boja, tocka)
        self.liste = liste

    def Draw(self, Canvas):
        Canvas.create_polygon(self.liste, outline=self.boja, fill="")


class Kruznica(Graflik):
    def __init__(self, boja, tocka, r):
        Graflik.__init__(self, boja, tocka)
        self.r = float(r)  

    def Draw(self, Canvas):
        Canvas.create_oval(self.tocka[0] - self.r, self.tocka[1] - self.r,
                           self.tocka[0] + self.r, self.tocka[1] + self.r, outline=self.boja, fill="")



class Elipsa(Kruznica):
    def __init__(self, boja, tocka, r_x, r_y):
        Kruznica.__init__(self, boja, tocka, 0.2)
        self.r_x = float(r_x)
        self.r_y = float(r_y)

    def Draw(self, Canvas):
        Canvas.create_oval(self.tocka[0] - self.r_x, self.tocka[1] - self.r_y,
                           self.tocka[0] + self.r_x, self.tocka[1] + self.r_y, outline=self.boja, fill="")


class ApplicationDrawing(Frame):
    def createCanvas(self):
        self.canvas = Canvas(self, bg="#999999", height=600, width=800)
        self.canvas.pack()
        self.server_status_label = Label(self, text="")
       
        self.server_status_label.pack(anchor=E)


    def __init__(self, master=None):
        Frame.__init__(self, master)
        self.master = master

        menu = Menu(self.master)
        self.master.config(menu=menu)
        file_menu = Menu(menu)
        file_menu.add_command(label="Open", command=self.OpenFile)
        file_menu.add_command(label="Exit", command=self.quit)
        menu.add_cascade(label="File", menu=file_menu)
        server_menu = Menu(menu, tearoff=0)
        server_menu.add_command(label="Start Server", command=self.start_server)
        menu.add_cascade(label="Server", menu=server_menu)

        self.pack()
        self.createCanvas()

    def start_server(self):
        def handle_client(conn):
            while True:
                data = conn.recv(1024)
                if not data:
                    break

        # Parse data as string and split by comma.
                shape_data = data.decode('utf-8').split(',')

                shape_type = shape_data[0]
                color = shape_data[1]

        # Handling each shape according to its type.
                if shape_type == 'Line':
                    point1_x = float(shape_data[2])
                    point1_y = float(shape_data[3])
                    point2_x = float(shape_data[4])
                    point2_y = float(shape_data[5])
                    line = Linija(color, [point1_x, point1_y], [point2_x, point2_y])
                    line.Draw(self.canvas)

                elif shape_type == 'Rectangle':
                    point_x = float(shape_data[2])
                    point_y = float(shape_data[3])
                    width = float(shape_data[4])
                    height = float(shape_data[5])
                    rectangle = Pravokutnik(color, (point_x, point_y), height, width)
                    rectangle.Draw(self.canvas)

                elif shape_type == 'Triangle':
                    point1_x = float(shape_data[2])
                    point1_y = float(shape_data[3])
                    point2_x = float(shape_data[4])
                    point2_y = float(shape_data[5])
                    point3_x = float(shape_data[6])
                    point3_y = float(shape_data[7])
                    triangle = Trokut(color, (point1_x, point1_y), (point2_x, point2_y), (point3_x, point3_y))
                    triangle.Draw(self.canvas)

                elif shape_type == 'Circle':
                    center_x = float(shape_data[2])
                    center_y = float(shape_data[3])
                    radius = float(shape_data[4])
                    circle = Kruznica(color, (center_x, center_y), radius)
                    circle.Draw(self.canvas)

                elif shape_type == 'Ellipse':
                    center_x = float(shape_data[2])
                    center_y = float(shape_data[3])
                    radius_x = float(shape_data[4])
                    radius_y = float(shape_data[5])
                    ellipse = Elipsa(color, (center_x, center_y), radius_x, radius_y)
                    ellipse.Draw(self.canvas)

                elif shape_type == 'Polygon':
                    # Parse points from shape_data[2:] into pairs of (x, y).
                    points = [(float(shape_data[i]), float(shape_data[i + 1])) for i in range(2, len(shape_data), 2)]
                    polygon = Poligon(color, points[0], points[1:])
                    polygon.Draw(self.canvas)

            conn.close()

       

        def server_thread():
            global clients
            hostname = socket.gethostname()
            port = 8080
            port1 = str(port)
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                s.bind(('localhost', port))
                print("Server is running and ready to receive connections.")
                # Update server status label.
                
                self.server_status_label['text'] = "Server is running on host" + hostname + " on port " + port1

                s.listen(10)
                while True:
                    if len(clients) < CLIENT_LIMIT:
                        conn, addr = s.accept()
                        clients.append(conn)
                        print(f"Connection established with {addr}")
                        threading.Thread(target=handle_client, args=(conn,)).start()
                    else:
                        print(f"Server is full, can't accept new connections.")

        threading.Thread(target=server_thread).start()




    def OpenFile(self):
        filename = filedialog.askopenfilename()
        txt_file = open(filename, 'r')
        lists = []
        for line in txt_file:
            strpdLine = line.strip()
            list_lines = strpdLine.split()
            if len(list_lines) >= 5:
                list_lines = (list_lines[0], list_lines[1],
                              *(float(v) for v in list_lines[2:]))
                lists.append(list_lines)

        for line in lists:
            if line[0] == "Line":
                linija1 = Linija(line[1], line[2:4], line[4:6])
                linija1.Draw(self.canvas)
            elif line[0] == "Triangle":
                trokut1 = Trokut(line[1], line[2:4], line[4:6], line[6:8])
                trokut1.Draw(self.canvas)
            elif line[0] == "Rectangle":
                pravokutnik1 = Pravokutnik(
                    line[1], line[2:4], line[4], line[5])
                pravokutnik1.Draw(self.canvas)
            elif line[0] == "Polygon":
                poligon1 = Poligon(line[1], line[2:4], line[4:])
                poligon1.Draw(self.canvas)
            elif line[0] == "Circle":
                kruznica1 = Kruznica(line[1], line[2:4], line[4])
                kruznica1.Draw(self.canvas)
            elif line[0] == "Ellipse":
                elipsa1 = Elipsa(line[1], line[2:4], line[4], line[5])
                elipsa1.Draw(self.canvas)



def main():
    root = Tk()
    aplikacija = ApplicationDrawing(root)
    root.wm_title("Drawing App v1.0 Davorin")
    aplikacija.mainloop()


main()

Server is running and ready to receive connections.


In [2]:
def main():
    root = Tk()
    aplikacija = ApplicationDrawing(root)
    root.wm_title("Drawing App v1.0 Davorin")
    aplikacija.mainloop()


In [3]:

main()


Server is running and ready to receive connections.
